In [2]:
%load_ext autoreload  
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import pypianoroll
import matplotlib.pyplot as plt
import pandas as pd


In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

In [5]:
import IPython
import pretty_midi
from midi2audio import FluidSynth
import os
import shutil
import glob
import pretty_midi
import pickle
from music21 import converter, instrument, note, chord, stream
import music21
# import librosa
import matplotlib.pyplot as plt
# from tensorflow.keras.utils import np_utils
import json
import IPython.display
from datetime import datetime
import mido
root_dir = '../raw_data'
from music_generator.data import get_npz_data, multitrack_to_midi, fetching_instrument_pianorolls
from music_generator.midi import sample_multitrack

In [15]:
data = get_npz_data(1000, data_path='../raw_data/lpd_5/lpd_5_cleansed')

In [ ]:
midi_transfo = multitrack_to_midi(data, save_path='../raw_data/midi')

In [16]:
pianorolls = fetching_instrument_pianorolls(data, 'Piano')

In [ ]:
pianorolls[0]

In [ ]:
test = []
resolution = 24
for i in pianorolls:
    test.append(sample_multitrack(data[0], ))

In [ ]:
def sample_multitrack(multitrack, track, x_qnotes=4, y_qnotes=2):
#     pianoroll = track.pianoroll
    qnote = multitrack.resolution
#     quarter_notes = int(pianoroll.shape[0]/qnote)
    X = []
    y = []
    for i in range(int(np.floor(quarter_notes/(x_qnotes+y_qnotes)))):
        X_start = i*x_qnotes *qnote
        X_end = ((i+1)*x_qnotes) *qnote
        y_start = X_end
        y_end = ((i+1)*x_qnotes + y_qnotes) *qnote
        X.append(pianoroll[X_start:X_end])
        y.append(pianoroll[y_start:y_end])
    return np.array(X), np.array(y)

In [ ]:
test_x = sample_multitrack(data[0], pianorolls_dumb)

In [ ]:
## trying sampling without func
X = []
y = []
qnote = 24
x_qnotes=8
y_qnotes=4
for pianoroll in pianorolls:
    quarter_notes = int(pianoroll.shape[0]/qnote)
    for i in range(int(np.floor(quarter_notes/(x_qnotes+y_qnotes)))):
        X_start = i*x_qnotes *qnote
        X_end = ((i+1)*x_qnotes) *qnote
        y_start = X_end
        y_end = ((i+1)*x_qnotes + y_qnotes) *qnote
        X.append(pianoroll[X_start:X_end])
        y.append(pianoroll[y_start:y_end])
X=np.array(X)
y=np.array(y)


In [19]:
## sampling multiple tracks with func - One Array
def multi_sample_pianorolls(pianorolls, qnote = 24, x_qnotes = 192, y_qnotes = 192):
    '''pianorolls = output of 'fetching_instrument_pianorolls' function in data.py. 
    Returns an array concatenating all the samples of n songs.'''
    X = []
    y = []
    for pianoroll in pianorolls:
        quarter_notes = int(pianoroll.shape[0]/qnote)
        for i in range(int(np.floor(quarter_notes/(x_qnotes+y_qnotes)))):
            X_start = i*x_qnotes *qnote
            X_end = ((i+1)*x_qnotes) *qnote
            y_start = X_end
            y_end = ((i+1)*x_qnotes + y_qnotes) *qnote
            X.append(pianoroll[X_start:X_end])
            y.append(pianoroll[y_start:y_end])
    return np.array(X), np.array(y)

In [20]:
X, y = multi_sample_pianorolls(pianorolls)

In [22]:
X.shape

(398, 4608, 128)

In [23]:
def init_model(X, y):
    input_shape = X[0].shape
    output_shape = y[0].shape[1]
    print (input_shape, output_shape)
    model = models.Sequential()
    model.add(layers.SimpleRNN(128, return_sequences=True, activation='relu', input_shape=input_shape))
    model.add(layers.SimpleRNN(128, return_sequences=True, activation='tanh', input_shape=input_shape))
    model.add(layers.SimpleRNN(128, return_sequences=True, activation='tanh', input_shape=input_shape))
    model.add(layers.SimpleRNN(128, return_sequences=True, activation='relu'))
    model.add(layers.Dense(128, activation='tanh'))
    model.compile(loss='mse', optimizer=RMSprop(learning_rate=0.001))
    return model

model = init_model(X, y)
model.summary()

(4608, 128) 128
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4608, 128)         32896     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 4608, 128)         32896     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 4608, 128)         32896     
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 4608, 128)         32896     
_________________________________________________________________
dense (Dense)                (None, 4608, 128)         16512     
Total params: 148,096
Trainable params: 148,096
Non-trainable params: 0
_________________________________________________________________


2021-11-24 11:12:10.007894: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 2, restore_best_weights=True)

history = model.fit(X, y, validation_split=0.2, epochs=10, batch_size=16)

2021-11-24 11:12:19.720292: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
20/20 [==============================] - 79s 4s/step - loss: 128.3334 - val_loss: 141.3727
Epoch 2/10
20/20 [==============================] - 77s 4s/step - loss: 127.8924 - val_loss: 141.0768
Epoch 3/10
20/20 [==============================] - 77s 4s/step - loss: 127.7213 - val_loss: 141.0610
Epoch 4/10
20/20 [==============================] - 78s 4s/step - loss: 127.7196 - val_loss: 141.0306
Epoch 5/10
20/20 [==============================] - 77s 4s/step - loss: 127.7024 - val_loss: 141.0418
Epoch 6/10
20/20 [==============================] - 77s 4s/step - loss: 127.6984 - val_loss: 141.0502
Epoch 7/10
20/20 [==============================] - 77s 4s/step - loss: 127.6958 - val_loss: 141.0300
Epoch 8/10
20/20 [==============================] - 76s 4s/step - loss: 127.6895 - val_loss: 141.0278
Epoch 9/10
20/20 [==============================] - 75s 4s/step - loss: 127.6845 - val_loss: 141.0186
Epoch 10/10
20/20 [==============================] - 76s 4s/step - loss: 127.6771 

In [26]:
pred = model.predict(X)
pred.astype(np.uint8)


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [28]:
pred.shape

(398, 4608, 128)

In [ ]:
from music_generator.midi import play_pianoroll

play_pianoroll(data[0], pred[:15].reshape(15*4608, 128))